# TODO

* [ ] Perceber problema 
* [ ] Ler e perceber o quê CRISP DM - Methodology
* [ ] Entender Dataset
* [ ] Integrar Dataset aqui
* [ ] Perceber que métodos vamos fazer
* [ ] Comentar resultados 
* [ ] Começar Word 


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

cvs_health = pd.read_csv('CVD_cleaned.csv')
print(cvs_health)

       General_Health                  Checkup Exercise Heart_Disease  \
0                Poor  Within the past 2 years       No            No   
1           Very Good     Within the past year       No           Yes   
2           Very Good     Within the past year      Yes            No   
3                Poor     Within the past year      Yes           Yes   
4                Good     Within the past year       No            No   
...               ...                      ...      ...           ...   
308849      Very Good     Within the past year      Yes            No   
308850           Fair  Within the past 5 years      Yes            No   
308851      Very Good      5 or more years ago      Yes            No   
308852      Very Good     Within the past year      Yes            No   
308853      Excellent     Within the past year      Yes            No   

       Skin_Cancer Other_Cancer Depression  \
0               No           No         No   
1               No           No

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, classification_report, silhouette_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Funções de Data Understanding e Preparation permanecem iguais...

# Supervised Learning Atualizado
def train_supervised_models(X_train, X_test, y_train, y_test):
    models = {
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42),
        'AdaBoost': AdaBoostClassifier(random_state=42),
        'MLP': MLPClassifier(random_state=42, max_iter=1000),
        'KNN': KNeighborsClassifier(),
        'SVM': SVC(random_state=42),
        'Naive Bayes': GaussianNB(),
        'Logistic Regression': LogisticRegression(random_state=42)
    }
    
    results = {}
    for name, model in models.items():
        print(f"\nTreinando {name}...")
        
        # Treinar e fazer cross-validation
        cv_scores = cross_val_score(model, X_train, y_train, cv=5)
        
        # Treinar no conjunto completo
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # Métricas
        accuracy = accuracy_score(y_test, y_pred)
        conf_matrix = confusion_matrix(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        
        results[name] = {
            'cv_scores_mean': cv_scores.mean(),
            'cv_scores_std': cv_scores.std(),
            'test_accuracy': accuracy,
            'confusion_matrix': conf_matrix,
            'classification_report': report,
            'model': model
        }
        
        print(f"CV Accuracy: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
        print(f"Test Accuracy: {accuracy:.4f}")
    
    return results

# Unsupervised Learning Atualizado
def perform_clustering(features_scaled):
    clustering_results = {}
    
    # K-Means
    print("\nExecutando K-Means...")
    kmeans = KMeans(n_clusters=3, random_state=42)
    kmeans_labels = kmeans.fit_predict(features_scaled)
    kmeans_silhouette = silhouette_score(features_scaled, kmeans_labels)
    clustering_results['KMeans'] = {
        'labels': kmeans_labels,
        'silhouette': kmeans_silhouette,
        'model': kmeans
    }
    
    # DBSCAN
    print("Executando DBSCAN...")
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    dbscan_labels = dbscan.fit_predict(features_scaled)
    if len(set(dbscan_labels)) > 1:
        dbscan_silhouette = silhouette_score(features_scaled, dbscan_labels)
    else:
        dbscan_silhouette = None
    clustering_results['DBSCAN'] = {
        'labels': dbscan_labels,
        'silhouette': dbscan_silhouette,
        'model': dbscan
    }
    
    # Agglomerative Clustering
    print("Executando Agglomerative Clustering...")
    agglo = AgglomerativeClustering(n_clusters=3)
    agglo_labels = agglo.fit_predict(features_scaled)
    agglo_silhouette = silhouette_score(features_scaled, agglo_labels)
    clustering_results['Agglomerative'] = {
        'labels': agglo_labels,
        'silhouette': agglo_silhouette,
        'model': agglo
    }
    
    # Spectral Clustering
    print("Executando Spectral Clustering...")
    spectral = SpectralClustering(n_clusters=3, random_state=42)
    spectral_labels = spectral.fit_predict(features_scaled)
    spectral_silhouette = silhouette_score(features_scaled, spectral_labels)
    clustering_results['Spectral'] = {
        'labels': spectral_labels,
        'silhouette': spectral_silhouette,
        'model': spectral
    }
    
    # Gaussian Mixture
    print("Executando Gaussian Mixture...")
    gmm = GaussianMixture(n_components=3, random_state=42)
    gmm_labels = gmm.fit_predict(features_scaled)
    gmm_silhouette = silhouette_score(features_scaled, gmm_labels)
    clustering_results['GMM'] = {
        'labels': gmm_labels,
        'silhouette': gmm_silhouette,
        'model': gmm
    }
    
    return clustering_results

# Visualização dos resultados
def plot_clustering_results(features_scaled, clustering_results):
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.ravel()
    
    # Usar PCA para visualização 2D
    from sklearn.decomposition import PCA
    pca = PCA(n_components=2)
    features_2d = pca.fit_transform(features_scaled)
    
    for idx, (name, results) in enumerate(clustering_results.items()):
        axes[idx].scatter(features_2d[:, 0], features_2d[:, 1], c=results['labels'], cmap='viridis')
        axes[idx].set_title(f'{name}\nSilhouette: {results["silhouette"]:.3f}')
    
    plt.tight_layout()
    plt.show()

# Função principal atualizada
def main():
    # Carregar e preparar dados
    df = load_and_explore_data('cardiovascular_data.csv')
    features_scaled, target = prepare_data(df)
    
    # Split dos dados
    X_train, X_test, y_train, y_test = train_test_split(
        features_scaled, target, test_size=0.2, random_state=42
    )
    
    # Treinar e avaliar modelos supervisionados
    print("\nTreinando modelos supervisionados...")
    supervised_results = train_supervised_models(X_train, X_test, y_train, y_test)
    
    # Realizar clustering
    print("\nRealizando clustering...")
    clustering_results = perform_clustering(features_scaled)
    plot_clustering_results(features_scaled, clustering_results)
    
    # Avaliar impacto de valores ausentes
    print("\nAvaliando impacto de valores ausentes...")
    missing_values_results = evaluate_missing_values_impact(df)
    
    return {
        'supervised_results': supervised_results,
        'clustering_results': clustering_results,
        'missing_values_results': missing_values_results
    }

if __name__ == "__main__":
    results = main()


KeyboardInterrupt: 

In [3]:
cvs_health.head()

,General_Health,Checkup,Exercise,Heart_Disease,Skin_Cancer,Other_Cancer,Depression,Diabetes,Arthritis,Sex,Age_Category,Height_(cm),Weight_(kg),BMI,Smoking_History,Alcohol_Consumption,Fruit_Consumption,Green_Vegetables_Consumption,FriedPotato_Consumption
0,Poor,Within the past 2 years,No,No,No,No,No,No,Yes,Female,70-74,150.0,32.66,14.54,Yes,0.0,30.0,16.0,12.0
1,Very Good,Within the past year,No,Yes,No,No,No,Yes,No,Female,70-74,165.0,77.11,28.29,No,0.0,30.0,0.0,4.0
2,Very Good,Within the past year,Yes,No,No,No,No,Yes,No,Female,60-64,163.0,88.45,33.47,No,4.0,12.0,3.0,16.0
3,Poor,Within the past year,Yes,Yes,No,No,No,Yes,No,Male,75-79,180.0,93.44,28.73,No,0.0,30.0,30.0,8.0
4,Good,Within the past year,No,No,No,No,No,No,No,Male,80+,191.0,88.45,24.37,Yes,0.0,8.0,4.0,0.0
